In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Cập nhật đường dẫn tệp để phản ánh đường dẫn mount Google Drive chính xác
df_main = pd.read_csv('/content/drive/MyDrive/Machine learning/annonimized.csv')
df_th = pd.read_csv('/content/drive/MyDrive/Machine learning/th-public.csv')

print(f" Main data: {len(df_main):,} records")
print(f" QT scores: {len(df_th):,} students")

 Main data: 295,198 records
 QT scores: 761 students


In [ ]:
# Khám phá cấu trúc dữ liệu chính
print(" Cấu trúc file annonimized.csv:")
print(df_main.info())
print("\n Sample data:")
df_main.head()

 Cấu trúc file annonimized.csv:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295198 entries, 0 to 295197
Data columns (total 11 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   concat('it001',`assignment_id`)  295198 non-null  object
 1   concat('it001',`problem_id`)     295198 non-null  object
 2   concat('it001', username)        295198 non-null  object
 3   is_final                         295198 non-null  int64 
 4   status                           295198 non-null  object
 5   pre_score                        295198 non-null  int64 
 6   coefficient                      295198 non-null  int64 
 7   concat('it001',`language_id`)    295198 non-null  object
 8   created_at                       295198 non-null  object
 9   updated_at                       295198 non-null  object
 10  judgement                        295198 non-null  object
dtypes: int64(3), object(8)
memory usage: 24.8+ MB


,"concat('it001',`assignment_id`)","concat('it001',`problem_id`)","concat('it001', username)",is_final,status,pre_score,coefficient,"concat('it001',`language_id`)",created_at,updated_at,judgement
0,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,10-09 08:02:04,10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
1,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,10-09 08:04:41,10-09 08:04:51,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
2,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 08:06:49,10-09 08:06:58,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
3,90ce27571176d87961b565d5ef4b3de33ede04ac,bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 08:47:52,10-09 08:48:01,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."
4,90ce27571176d87961b565d5ef4b3de33ede04ac,7a6e5ca470ff47c3b5048f240c4738de71010c78,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,10-09 09:19:35,10-09 09:19:45,"{""times"":[0,0,0,0,0,0,0,0,0,0],""mems"":[0,0,0,0..."


In [ ]:
# Đổi tên các cột
# Thay thế df bằng df_main vì dataframe được tải có tên là df_main
df_main.columns = ['assignment_id', 'problem_id', 'mssv', 'is_final', 'status',
              'pre_score', 'late_coef', 'language_id', 'created_at', 'updated_at', 'judgement']

# Bỏ luôn 2 cột thời gian
# Thay thế df bằng df_main
df_main = df_main.drop(columns=['created_at', 'updated_at'])

In [ ]:
 agg_df = df_main.groupby("mssv").agg({
    'assignment_id': 'nunique',
    'problem_id': 'nunique',
    'is_final': 'sum',
    'status': lambda x: (x == 'passed').sum(),  # số bài passed
    'pre_score': ['mean', 'max', 'std'],
    'late_coef': 'mean',
}).fillna(0)

agg_df.columns = ['_'.join(col) for col in agg_df.columns]
agg_df = agg_df.reset_index()

print(" Số sinh viên tổng hợp được:", len(agg_df))
agg_df.info()

 Số sinh viên tổng hợp được: 1489
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   mssv                   1489 non-null   object 
 1   assignment_id_nunique  1489 non-null   int64  
 2   problem_id_nunique     1489 non-null   int64  
 3   is_final_sum           1489 non-null   int64  
 4   status_<lambda>        1489 non-null   int64  
 5   pre_score_mean         1489 non-null   float64
 6   pre_score_max          1489 non-null   int64  
 7   pre_score_std          1489 non-null   float64
 8   late_coef_mean         1489 non-null   float64
dtypes: float64(3), int64(5), object(1)
memory usage: 104.8+ KB


In [ ]:
df_th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 761 entries, 0 to 760
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   hash    761 non-null    object
 1   TH      755 non-null    object
dtypes: object(2)
memory usage: 12.0+ KB


In [ ]:
df_th = df_th.rename(columns={'hash': 'mssv', 'TH': 'th'})

# Xử lý ký tự lỗi và ép kiểu float
df_th['th'] = df_th['th'].astype(str).str.replace('\xa0', '', regex=False)
df_th['th'] = pd.to_numeric(df_th['th'], errors='coerce')

In [ ]:
# Thao tác merge hai bảng
data = agg_df.merge(df_th, on='mssv', how='left')

NotNull_df = data[~data['th'].isna()]
Null_df = data[data['th'].isna()]

print(f"📚 Null samples: {len(NotNull_df)}, 🔍 NotNull samples: {len(Null_df)}")
data.head()


📚 Null samples: 753, 🔍 NotNull samples: 736


,mssv,assignment_id_nunique,problem_id_nunique,is_final_sum,status_<lambda>,pre_score_mean,pre_score_max,pre_score_std,late_coef_mean,th
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,7,46,46,0,5504.149660,10000,4541.435779,100.000000,5.0
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,9,78,78,0,5488.552124,10000,4323.760900,100.000000,8.5
2,01122b3ef7e59b84189e65985305f575d6bdf83c,7,66,67,0,5973.753846,10000,4169.936606,100.000000,7.0
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,4,47,47,0,5952.760000,10000,4640.302069,100.000000,NaN
4,013de369c439ab0ead8aa7da64423aa395a8be39,8,52,66,0,6474.448598,10000,4409.665925,97.196262,10.0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1489 entries, 0 to 1488
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   mssv                   1489 non-null   object 
 1   assignment_id_nunique  1489 non-null   int64  
 2   problem_id_nunique     1489 non-null   int64  
 3   is_final_sum           1489 non-null   int64  
 4   status_<lambda>        1489 non-null   int64  
 5   pre_score_mean         1489 non-null   float64
 6   pre_score_max          1489 non-null   int64  
 7   pre_score_std          1489 non-null   float64
 8   late_coef_mean         1489 non-null   float64
 9   th                     753 non-null    float64
dtypes: float64(4), int64(5), object(1)
memory usage: 116.5+ KB


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

X = NotNull_df.drop(columns=['mssv', 'th'])
y = NotNull_df['th']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Kích thước tập huấn luyện: {X_train.shape[0]} hàng, {X_train.shape[1]} cột")
print(f"Kích thước tập kiểm tra: {X_test.shape[0]} hàng, {X_test.shape[1]} cột")

Kích thước tập huấn luyện: 602 hàng, 8 cột
Kích thước tập kiểm tra: 151 hàng, 8 cột


In [ ]:
model = RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_split=5, min_samples_leaf=1, random_state=42)
model.fit(X_train, y_train)


RandomForestRegressor(max_depth=10, min_samples_split=5, random_state=42)

In [ ]:
test_preds = model.predict(X_test)

mse = mean_squared_error(y_test, test_preds)
print(f"MSE trên tập huấn luyện: {mse:.4f}")

mae = mean_absolute_error(y_test, test_preds)
print(f"MAE trên tập huấn luyện: {mae:.4f}")

r2 = r2_score(y_test, test_preds)
print(f"R² trên tập huấn luyện: {r2:.4f}")

MSE trên tập huấn luyện: 3.2318
MAE trên tập huấn luyện: 1.3738
R² trên tập huấn luyện: 0.3633


In [ ]:
# Chọn các cột đặc trưng từ Null_df tương ứng với các cột dùng để huấn luyện mô hình
X_null = Null_df[X_train.columns]

# Dự đoán điểm trên dữ liệu Null_df
null_preds = model.predict(X_null)

# Điền trực tiếp các giá trị dự đoán vào cột 'th' trong DataFrame data
data.loc[data['th'].isna(), 'th'] = null_preds

# Làm tròn điểm đã điền
data['th'] = data['th'].round(2)

In [ ]:
# Tách cột 'mssv' và 'th' từ DataFrame data
submission_df = data[['mssv', 'th']]

submission_df.to_csv("th_score.csv", index=False,header=False)
print("✅ Đã lưu file th_score.csv")

✅ Đã lưu file th_score.csv
